In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as ex
import plotly.graph_objs as go
import plotly.offline as pyo
from plotly.subplots import make_subplots
import keras
import tensorflow

In [ ]:
data = pd.read_csv('healthcare-dataset-stroke-data.csv')
print(data.shape)
data.head()

In [ ]:
data = data.drop(labels="id", axis=1)
data.head()

In [ ]:
data.isnull().sum().sort_values(ascending=False)

201 missing entries from BMI so we take the mean and fill in the missing rows.

In [ ]:
data.fillna(data.bmi.mean(), inplace=True)
data.isnull().sum().sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(9,9))
sns.heatmap(data.corr())
plt.show()

In [ ]:
sns.countplot(x=data['stroke'])
plt.show()

In [ ]:
fig = ex.pie(data, names='stroke')
fig.update_layout(title='<b>Stroke Proportion of Dataset')
fig.show()

stroke = 0
notStroke = 0

for i in data.stroke:
    if(i == 0):
        notStroke += 1
    elif(i == 1):
        stroke += 1
        
print(stroke, "people have stroke")
print(notStroke, "people don't have stroke")

95.1% of the patients sampled do not have stroke creating a highly skewed dataset.

In [ ]:
fig = ex.pie(data, names='gender')
fig.update_layout(title='<b>Gender Proportion of Dataset')
fig.show()

numMales = 0
numFemales = 0
other = 0


for i in data.gender:
    if(i == "Male"):
        numMales += 1
    if(i == "Female"):
        numFemales += 1
        
print(numMales, "Males")
print(numFemales, "Females")

In [ ]:
fig = ex.histogram(x=data.query('gender=="Male"').age)
fig.update_layout(title='<b>Male Proportion of Dataset')
fig.show()

In [ ]:
fig = ex.histogram(x=data.query('gender=="Female"').age)
fig.update_layout(title='<b>Female Proportion of Dataset')
fig.show()

In [ ]:
fig = ex.pie(data, names='smoking_status')
fig.update_layout(title='<b>Smokers Proportion of Dataset')
fig.show()

unknown = 0
neverSmoked = 0
smoke = 0
formerlySmoked = 0

for i in data.smoking_status:
    if(i == "Unknown"):
        unknown += 1
    if(i == "never smoked"):
        neverSmoked += 1
    if(i == "smokes"):
        smoke += 1
    elif(i == "formerly smoked"):
        formerlySmoked += 1

print(unknown, "people have an unknown smoking status")
print(neverSmoked, "people never smoked")
print(smoke, "people currently smoke")
print(formerlySmoked, "people formerly smoked")

In [ ]:
data1 = data.iloc[:,[0,2,3,4,5,6,9]]
data2 = data.iloc[:,[1,7,8]]
y = data.iloc[:,[-1]]

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
ohe = OneHotEncoder()
sc = StandardScaler()

In [ ]:
data1 = pd.DataFrame(ohe.fit_transform(data1).toarray())
data2 = pd.DataFrame(sc.fit_transform(data2))

In [ ]:
data = pd.concat([data1, data2], axis=1)
data.columns = [a for a in range(0, len(data.columns))]

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, xtest, Ytrain, ytest = train_test_split(data, y, test_size=0.20)

Artificial Neural Network Model

In [ ]:
from keras.models  import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(23,)))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=0.00001), metrics=["accuracy"])
model.summary()

In [ ]:
from keras.callbacks import EarlyStopping
callBack= EarlyStopping(
            monitor='accuracy',
            min_delta=0.001,
            patience=40,
            mode='auto')

In [ ]:
model.fit(Xtrain, Ytrain, epochs = 1000, batch_size=512, validation_split=0.20, callbacks=callBack)
accuracy = model.evaluate(xtest, ytest)

In [ ]:
print("ANN Accuracy: %.4f" % accuracy[1])

Implement Convolutional Neural Network